In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,12818
2,Huelva,Confirmados PDIA 14 días,1411
3,Huelva,Tasa PDIA 14 días,"276,26418766385444"
4,Huelva,Confirmados PDIA 7 días,888
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,19
629,Municipio de Huelva sin especificar,Total Confirmados,176
630,Municipio de Huelva sin especificar,Curados,114


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  12818.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  3633.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 639 personas en los últimos 7 días 

Un positivo PCR cada 371 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,12818.0,888.0,1411.0,510743.0,173.864351,276.264188,149.0
Huelva-Costa,7370.0,553.0,904.0,288115.0,191.937247,313.763601,81.0
Condado-Campiña,3809.0,199.0,318.0,155057.0,128.339901,205.085872,51.0
Huelva (capital),3633.0,225.0,387.0,143663.0,156.616526,269.380425,47.0
Villalba del Alcor,167.0,58.0,63.0,3338.0,1737.567406,1887.357699,34.0
Isla Cristina,699.0,135.0,230.0,21264.0,634.875847,1081.640331,16.0
Sierra de Huelva-Andévalo Central,1465.0,117.0,150.0,67571.0,173.151204,221.988723,13.0
Calañas,59.0,14.0,16.0,2791.0,501.612325,573.271229,5.0
Cartaya,483.0,28.0,51.0,19974.0,140.182237,255.331932,5.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Encinasola,36.0,26.0,27.0,1305.0,1992.337165,2068.965517,0.0
Villalba del Alcor,167.0,58.0,63.0,3338.0,1737.567406,1887.357699,34.0
Isla Cristina,699.0,135.0,230.0,21264.0,634.875847,1081.640331,16.0
Almendro (El),24.0,3.0,8.0,826.0,363.196126,968.523002,0.0
Calañas,59.0,14.0,16.0,2791.0,501.612325,573.271229,5.0
Aroche,72.0,12.0,17.0,3073.0,390.497885,553.205337,0.0
Corteconcepción,21.0,2.0,3.0,548.0,364.963504,547.445255,0.0
Cortegana,148.0,22.0,25.0,4666.0,471.495928,535.790827,5.0
Alosno,64.0,16.0,20.0,3923.0,407.851134,509.813918,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Escacena del Campo,102.0,1.0,4.0,2284.0,43.782837,175.131349,0.0,0.250000
Villablanca,71.0,1.0,4.0,2848.0,35.112360,140.449438,0.0,0.250000
San Bartolomé de la Torre,53.0,2.0,8.0,3736.0,53.533191,214.132762,0.0,0.250000
Cumbres Mayores,20.0,1.0,3.0,1755.0,56.980057,170.940171,0.0,0.333333
Almendro (El),24.0,3.0,8.0,826.0,363.196126,968.523002,0.0,0.375000
Cerro de Andévalo (El),64.0,5.0,12.0,2364.0,211.505922,507.614213,0.0,0.416667
Moguer,458.0,29.0,67.0,22088.0,131.293010,303.332126,0.0,0.432836
Palos de la Frontera,281.0,9.0,20.0,11289.0,79.723625,177.163611,1.0,0.450000
Valverde del Camino,283.0,5.0,10.0,12820.0,39.001560,78.003120,0.0,0.500000
